# Ploidy Python Library

In [5]:
# n, (the 'haploid number') is the number of chromosomes in a gamete. equal to half of total chromosome count. Hence, 2n = total chromosomes.
# If the organism undergoes meiosis and non-disjunction occurs, n is doubled.
# c_n (the coefficient of n) corresponds to whether the organism is a gamete or not. once an organism undergoes meiosis, c_n = 1. After conception and prior to meiosis, c_n = 2.
# c_n has a value of 1 if the organism is a gamete (during and after Meiosis II) and 2 in any other case.
# the value of x corresponds to the number of chromosomes which form a complete set (basically, it is the number of chromosomes in a genome).
# two or more chromosomes are said to be homeologous if they are very similar, yet cannot synapse during meiosis I.
# the value of c_x corresponds to the number of chromosome sets that are homeologous. indicated by type of ploidy (tri-, tetra-, quadra-, penta-, hexa-)
# the value of c_x is divided by two during meiosis, assuming non-disjunction doesn't occur. if non-disjunction occurs, then c_x doesn't change.

# a genome has an x number, a species has an n number and c_x.

class Polyploidic_Assortment:
  def __init__(self, *args, **kwargs):
    self.assortment = args
    self.dominant_allele = 'default!'
    self.gametes=[[self.assortment[0],self.assortment[1]],[self.assortment[2],self.assortment[3]],[self.assortment[0],self.assortment[2]],[self.assortment[1],self.assortment[3]],[self.assortment[0],self.assortment[3]],[self.assortment[1],self.assortment[2]]]
    # determine dominant allele
    if 'dominant_allele' in kwargs: self.dominant_allele=kwargs['dominant_allele']
    else:
      for allele in self.assortment:
        if allele[0].isupper(): self.dominant_allele = allele # if all alleles are lowercase ???
      assert (self.dominant_allele != 'default!'),'Failed to create Polyploidic_Allele object. No dominant_allele provided, no alleles were uppercase.'
      print('Dominant allele "{}" selected due to uppercase. Alternatively, pass dominant_allele as keyword argument.'.format(self.dominant_allele))
    # count dominant alleles, initialize plexy
    num_dominant_alleles = self.assortment.count(self.dominant_allele)
    plexies = ['nulli','sim','du','tri','quadri']
    self.plexy = plexies[num_dominant_alleles] + 'plex'
    # if provided by user, assign phenotypes
    if 'dominant_phenotype' in kwargs: 
      self.dominant_phenotype=kwargs['dominant_phenotype']
    if 'recessive_phenotype' in kwargs: 
      self.recessive_phenotype=kwargs['recessive_phenotype']
  def cross(self, mate):
    products=[]
    for i in range(0,6):
      for j in range(0,6):
        products.append(Polyploidic_Assortment(self.gametes[i][0], self.gametes[i][1], mate.gametes[j][0], mate.gametes[j][1], dominant_allele=self.dominant_allele).plexy)
    plexies = []
    for plexy in products:
      if plexy not in plexies: plexies.append(plexy)
    ratio = {'nulliplex':0,'simplex':0,'duplex':0,'triplex':0,'quadriplex':0}
    for plexy in plexies:
      ratio[plexy] = products.count(plexy)
    return ratio

class Monoploid_Genome:
  def __init__(self, name, x=0):
    self.name = name
    self.x = x

class Species: # ASSUME for now that x must be the same for all genomes
  def __init__(self, *args, **kwargs):
    self.genome = []
    self.prefixes = ['ha','di','tri','tetra','penta','hexa']
    self.ploidy = self.prefixes[len(args)-1] + 'ploid' # initialize, modify later if necessary
    self.total_chromosomes=0
    if len(args) == 1: # OPTION 1: parameters are (one) Monoploid_Genome, total_chromosomes, and x (unless Monoploid_Genome object x != 0)
      assert ('total_chromosomes' in kwargs),'Species object cannot be created. Species __init__ assumes if only one genome is provided that total_chromosomes and x will be provided.'
      self.total_chromosomes = kwargs['total_chromosomes']
      if args[0].x == 0:
        assert ('x' in kwargs),'Species object cannot be created. x must be provided, either to Species __init__ or Monoploid_Genome __init__.'
        self.x = kwargs['x'] # x passed as parameter
      else: self.x = args[0].x # x was passed as parameter to Monoploid_Genome object, copy its value
      assert (self.total_chromosomes % self.x == 0),'Species object cannot be created. Total chromosomes must be a multiple of x.'
      self.c_x = int(self.total_chromosomes / self.x)
      self.ploidy = self.prefixes[self.c_x-1] + 'ploid'
    elif len(args) > 1: # OPTION 2: parameters are multiple Monoploid_Genome objects
      assert (args[0].x == 0),'Species object cannot be created. Not all Monoploid_Genome objects have an x value, therefore total_chromosomes must be passed to Species __init__.'
      self.x = args[0].x # arbitrarily define x to match the number of chromosomes in the first genome.
      all_x_provided=True
      for monoploid_genome in args:
        if (monoploid_genome.x == 0): all_x_provided=False
          #'"{}" genome has more or less chromosomes than the first genome belonging to this Species.'.format(monoploid_genome.name)
        self.genome.append(monoploid_genome)
      if not all_x_provided:
        assert ('total_chromosomes' in kwargs),'Species object cannot be created. Not all Monoploid_Genome objects have an x value, therefore total_chromosomes must be passed to Species __init__.'
      if 'total_chromosomes' in kwargs: self.total_chromosomes = kwargs['total_chromosomes']
      else: self.total_chromosomes = self.c_x * self.x
      if self.total_chromosomes % 2 != 0: print('total_chromosomes is odd. value of n (unless provided as parameter) will be calculated truncating decimals from value of (total_chromosomes / 2).')
    # initialize derivative variables
    if 'n' in kwargs: self.n = kwargs['n'] # haploid number
    else: self.n = self.total_chromosomes / 2
    self.c_n = 2 # coefficient belonging to n
    if self.ploidy == 'haploid': self.c_n = 1
    # initialize gametes
    self.gamete='Gamete: '
    self.gametesReduced = True # whether meiosis I disjunction occurs
    for monoploid_genome in self.genome:
      if self.gametesReduced: self.gamete += monoploid_genome.name
      else: self.gamete += (monoploid_genome.name *2)
    self.unreduced_gamete = 'Unreduced gamete: '
    for monoploid_genome in self.genome:
      self.unreduced_gamete += (monoploid_genome.name * 2)
    # make sure that all values are consistent with 2n == c_x*x == total_chromosomes. Always true in the case that none of the values are provided by user.
    assert ((self.c_x * self.x == self.total_chromosomes and self.x == 0) or 2 * self.n == self.total_chromosomes),'Species __init__: parameter(s) provided are inconsistent with expression "2n == (coefficient)*x == total_chromosomes".'
  def cross(self, other):
    return 'blah'
  def toString(self):
    return '{}, {}n = {}x = {}. (x={}, n={})'.format(self.ploidy, self.c_n, self.c_x, self.total_chromosomes, self.x, self.n)

# Homework 7

### Question 1
Cotton was cultivated for millennia as a source of fiber in the Old World. The two most commonly cultivated species were Gossypium herbacium and G. arboreum. Both species have 2n = 26 chromosomes. They differ from each other by a reciprocal translocation. Cotton was also cultivated by ancient civilizations in the New World. The two cultivated species were G. hirsutum and G. barbadense, each with 2n = 52 chromosomes. A total of 26 wild species with 2n = 26 chromosomes have been identified in Africa, South and Central America, Australia, and the Middle East. Two wild species with 2n = 52 chromosomes have been identified, one with a limited range in northern Brazil and a second in Hawaii.  What is the ploidy of the New World cotton cultivated cotton species G. hirsutum and G. barbadense? 

In [14]:
# Reorganize:
'''
old_cotton = Species(Monoploid_Genome(name='H'), total_chromosomes=26)
print('Old world cotton:', old_cotton.toString())

new_cotton = Species(Monoploid_Genome(name='H'), total_chromosomes=52)
print('New world cotton:', G_hirsutum.toString())
'''
# Solve:
# What is x?

# Assumptions:
# x is the same for all cottons
# all varieties in question have an even ploidy number

old_cotton = Species(Monoploid_Genome(name='H'), total_chromosomes=26, x=13)
print('Old world cotton:', old_cotton.toString())

new_cotton = Species(Monoploid_Genome(name='H'), total_chromosomes=52, x=13)
print('New world cotton:', G_hirsutum.toString())

# Answer: Tetraploid.
# Reasoning: New world varieties with 2n = 52 chromosomes are autotetraploids derived from old world varieties.

Old world cotton: diploid, 2n = 2x = 26. (x=13, n=13.0)
New world cotton: tetraploid, 2n = 4x = 52. (x=13, n=26.0)


## Context for questions 2 and 3
In alfalfa, an autotetraploid, purple flowers are due to an allele (C1) that shows complete dominance to the recessive c1 allele. White flowers appear when the plants are nulliplex (c1c1c1c1) whereas purple flowers will appear if at least one of the alleles is C1.  A purple-flowered plant is crossed to a white-flowered plant. Half of the F1 progeny are purple-flowered and half are white-flowered.



### Question 2
What is the genotype of the purple-flowered parent?

In [7]:
simplex_purple = Polyploidic_Assortment('C1','c1','c1','c1', dominant_allele='C1')
white = Polyploidic_Assortment('c1','c1','c1','c1', dominant_allele='C1')

# F1 progeny
# half are white = Polyploidic_Assortment('c1','c1','c1','c1', dominant_allele='C1')
# other half are purple, meaning at least one C1 allele each

# Assumptions:
# Purple parent isn't C1 x 4
print(simplex_purple.cross(white))

# Answer: in order to achieve 1:1 ratio from a cross between a white and a purple, the purple would have to be simplex

{'nulliplex': 18, 'simplex': 18, 'duplex': 0, 'triplex': 0, 'quadriplex': 0}


### Question 3
What proportion of purple-flowered progeny would be expected in the F2 generation if one of the purple-flowered F1 plants is self-pollinated? 

In [8]:
purplef1 = Polyploidic_Assortment('C1','c1','c1','c1', dominant_allele='C1') # purple f1 progeny are C1 simplex
print(purple.cross(purplef1))

# Answer: 18 simplex + 9 duplex -> 27 purple / 36 total = 0.75

NameError: ignored

# Context for questions 4, 5, and 6
Drosophila fruit flies are diploids (2n = 2x = 8).

## Question 4
How many total chromosomes would a monosomic diploid fly (2n - 1) have in a somatic cell: 

In [ ]:
# Assumptions
# monosomic means that one of its chromosomes is missing

# Answer: 2n = 8 -> 8 - 1 = 7

## Question 5
Suppose a monosomic chromosome IV female Drosophila fly that has white eyes (a recessive X-linked trait) and normal bristles is crossed with a male with a diploid set of chromosomes and normal red eye color but homozygous for the recessive chromosome IV bristle mutation shaven (sv).  In the F1 generation, what is the expected proportion of white-eyed, shaven-bristled progeny? 

In [ ]:
# Assumptions:

# Female: 
#    IV: S_
#     X: ww
# gametes: S,w ; _,w

# Male:
#    IV: ss
#     X: WY
# gametes: s,W ; s,Y

# F1 zygotes: Ss,Ww; Ss,wY; s_,Ww; s_,wY 

# Answer: proportion of ss(or s_), ww(or wY) (shaven-bristled, white-eyed progeny) is 1/4

## Question 6
Eyeless is a recessive gene on chromosome 4 of Drosophila melanogaster. Flies homozygous for ey have tiny eyes or no eyes at all.  A male fly trisomic for chromosome 4 with genotype +/+/ey is crossed with a normal, diploid eyeless female. 

What proportion of their progeny will be eyeless? 

In [ ]:
# Female: ey/ey
# Male +/+/ey

# male gametes  : +/+, ey, +/ey, +, +/ey, +
# female gametes: ey
# zygotes       : +/+/ey, ey/ey, 2(+/ey/ey), 2(+/ey),

# Answer: proportion of ey/ey (eyeless) progeny is 1/6

# Context for questions 7, 8 and 9
Perennial desert shrubs of the genus Atriplex are typically dioecious; they consist of separate male and female plants.  Species within this genus also have varying levels of ploidy.  Four-wing saltbush (A. canescens) and salt sage (A. tridentata) are wide-spread species throughout the American west.  Both are fully fertile.  In 1979, Stutz et al. (Amer. J. Bot. 66:1181-1193) reported the discovery of an isolated female four-wing saltbush plant surrounded by a large population of salt sage near the south shore of the Great Salt Lake.  Chromosomes counts in the somatic cells of the salt sage population consistently yielded 2n = 54.  Seed collected from the isolated female four-wing saltbush plant were grown in a garden.  The plants that arose from the seeds were sterile and had 45 chromosomes in the somatic cells.

## Question 7
How many chromosomes are in the somatic cells of the four-wing saltbush female plant?

In [ ]:
saltbush = Species(Monoploid_Genome(name='S'), total_chromosomes=36)
saltsage = Species(Monoploid_Genome(name='S'), total_chromosomes=54)

# saltsage n = 27
# progeny n = 45
# therefore, saltbush n = 45 - 27 = 18
# therefore, saltbush total_chromosomes = 36
# Answer:                                ^^^

## Question 8
What is the ploidy of the four-wing saltbush plant?

In [ ]:
# Assumptions:
# x is the same for both plants

# x must be a factor of the total number of chromosomes in both species as well as their hybrid
# x = 9 since 9 is the greatest common factor they all share (is this always true?).

saltbush = Species(Monoploid_Genome(name='S'), x=9, total_chromosomes=36)
saltsage = Species(Monoploid_Genome(name='S'), x=9, total_chromosomes=54)

print('Saltbush:', saltbush.toString())
print('Saltsage:', saltsage.toString())

# Answer: saltbush ploidy is tetraploid

## Question 9
What is the ploidy of the sterile progeny plants grown in the garden.

In [ ]:
progeny = Species(Monoploid_Genome(name='H'), x=9, total_chromosomes=45)
print('Progeny:', progeny.toString())

## Question 10
Mendelian ratios are modified in crosses involving autotetraploids.  In an autotetraploid pea species, green seed is governed by the dominant gene I (i.e., I--- plants have yellow seed, while iiii plants have green seed) and round seed shape is governed by the dominant gene R (i.e., R--- plants have round seed, while rrrr plants have wrinkled seed).   

What is the predicted proportion of green round seed in the F2 progeny of a “dihybrid” cross involving the two independently assorting characteristics (e.g., IIIIRRRR X iiiirrrr).

In [ ]:
# quasi-dihybrid cross

p1 = Polyploidic_Assortment(['I','I','I','I'],['R','R','R','R'])
p2 = Polyploidic_Assortment(['i','i','i','i'],['r','r','r','r'])

f1 = Polyploidic_Assortment(['I','I','i','i'],['R','R','r','r'])

# f1 gene 1 gametes: II ii 4*(Ii)
# f1 gene 2 gametes: RR rr 4*(Rr)
# smartest way of doing this: use independent assortment and product rule of probability.
# the probability of a particular combination of phenotypes (or genotypes) is equal to product the probabities of the individual events.
# the phenotype we want is green, round seeds. green seed is recessive, so nulliplex I. round seeds are dominant, so only one dominant allele is required.
# doing a punnett square considering only gene 1 shows that only 1 of the 36 products is nulliplex (iiii)
# we only need one punnett square b/c both sets of gametes are the same as far as zygosity
# therefore, the other 35 are non-nulliplex, meaning they have at least one dominant allele

f2 = Polyploidic_Assortment([])

# Answer: Proportion of green, round seed in the F2 progeny is 1/36 * 35/36 = 35/1296

# Code logic tests

## Zebroid test


In [ ]:
# zebroid test

mountain_zebra = Species(Monoploid_Genome(name='M'), Monoploid_Genome(name='M'), total_chromosomes=32)
plains_zebra = Species(Monoploid_Genome(name='P'), Monoploid_Genome(name='P'), total_chromosomes=44)
grévys_zebra = Species(Monoploid_Genome(name='G'), Monoploid_Genome(name='G'), total_chromosomes=46)
kiang = Species(Monoploid_Genome(name='K'), Monoploid_Genome(name='K'), total_chromosomes=52)
onager = Species(Monoploid_Genome(name='O'), Monoploid_Genome(name='O'), total_chromosomes=56)
domesticated_donkey = Species(Monoploid_Genome(name='D'), Monoploid_Genome(name='D'), total_chromosomes=62)
horse = Species(Monoploid_Genome(name='H'), Monoploid_Genome(name='H'), total_chromosomes=64)


## uneven diploid number test

In [ ]:
# certain numbers, when divided by 2, yield an odd number (6, 22, 34), and others yield even numbers (8, 16, 36)

species1 = Species(Monoploid_Genome(name='A'), Monoploid_Genome(name='A'), total_chromosomes=32)   # even value of 'n'
species2 = Species(Monoploid_Genome(name='B'), Monoploid_Genome(name='B'), total_chromosomes=34) # odd value of 'n'
print(species1.toString())
print(species2.toString())



## polyploidy tests

In [ ]:
gamete = Species(Monoploid_Genome(name='C', x=2)) # 2n = 2x = 4
print('Gamete:', gamete.toString(), gamete.gamete)

diploid1 = Species(Monoploid_Genome(name='A', x=2), Monoploid_Genome(name='A', x=2)) # 2n = 2x = 4
diploid2 = Species(Monoploid_Genome(name='B', x=2), Monoploid_Genome(name='B', x=2)) # 2n = 2x = 4
print('Diploid1:', diploid1.toString(), diploid1.gamete, diploid1.unreduced_gamete)

triploid = Species(Monoploid_Genome(name='B', x=2), Monoploid_Genome(name='B', x=2), Monoploid_Genome(name='A', x=2)) # 2n = 3x = 4
print('Triploid1:', triploid.toString(), triploid.gamete)

# allotetraploid (allo- is a prefix which means "different" or "other")
# allotetraploid = diploid1.cross(diploid2)

# autotetraploid (auto- is a prefix which means "self" or "same")
# allotetraploid = diploid1.cross(diploid1)

# Bread wheat (AABBDD)
# 2n = 6x = 42 (n=21, x=7)
# Gamete is ABD

# Durum wheat (AABB)
# 2n = 4x = 28
# Gamete is AB

# cross the two and get pentaploid (AABBD)
# sterile due to inablity to synapse